[![下载Notebook](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/r2.0.0-alpha/tutorials/zh_cn/advanced/modules/mindspore_loss.ipynb)&emsp;[![下载样例代码](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/r2.0.0-alpha/tutorials/zh_cn/advanced/modules/mindspore_loss.py)&emsp;[![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/r2.0.0-alpha/tutorials/source_zh_cn/advanced/modules/loss.ipynb)

# 损失函数

损失函数，亦称目标函数，用于衡量预测值与真实值差异的程度。

在深度学习中，模型训练就是通过不断迭代来缩小损失函数值的过程。因此，在模型训练过程中损失函数的选择非常重要，一个好的损失函数能有效提升模型的性能。

`mindspore.nn`模块中提供了许多[通用损失函数](https://www.mindspore.cn/docs/zh-CN/r2.0.0-alpha/api_python/mindspore.nn.html#损失函数)，但这些通用损失函数无法满足所有需求，很多情况需要用户自定义所需的损失函数。因此，本教程介绍如何自定义损失函数。

![lossfun.png](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/tutorials/source_zh_cn/advanced/modules/images/loss_function.png)

## 内置损失函数

首先介绍`mindspore.nn`模块中内置的[损失函数](https://www.mindspore.cn/docs/zh-CN/r2.0.0-alpha/api_python/mindspore.nn.html#损失函数)。

下例以`nn.L1Loss`为例，计算预测值和目标值之间的平均绝对误差：

$$\ell(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad \text{with } l_n = \left| x_n - y_n \right|$$

其中N为数据集中的`batch_size`值。

$$\ell(x, y) =
        \begin{cases}
            \operatorname{mean}(L), & \text{if reduction} = \text{'mean';}\\
            \operatorname{sum}(L),  & \text{if reduction} = \text{'sum'.}
        \end{cases}$$

`nn.L1Loss`中的参数`reduction`取值可为`mean`，`sum`，或`none`。若`reduction` 为`mean`或`sum`，则输出一个经过均值或求和后的标量Tensor（降维）；若`reduction`为`none`，则所输出Tensor的shape为广播后的shape。

In [1]:
import numpy as np
from mindspore import nn
from mindspore import Tensor

# 输出loss均值
loss = nn.L1Loss()
# 输出loss和
loss_sum = nn.L1Loss(reduction='sum')
# 输出loss原值
loss_none = nn.L1Loss(reduction='none')

input_data = Tensor(np.array([[1, 2, 3], [2, 3, 4]]).astype(np.float32))
target_data = Tensor(np.array([[0, 2, 5], [3, 1, 1]]).astype(np.float32))

print("loss:", loss(input_data, target_data))
print("loss_sum:", loss_sum(input_data, target_data))
print("loss_none:", loss_none(input_data, target_data))

loss: 1.5
loss_sum: 9.0
loss_none: [[1. 0. 2.]
 [1. 2. 3.]]


## 自定义损失函数

自定义损失函数的方法有两种：一是基于`nn.Cell`来定义损失函数；二是`nn.LossBase`来定义损失函数。`nn.LossBase`继承自`nn.Cell`，额外提供了`get_loss`方法，利用`reduction`参数对损失值求和或求均值，输出一个标量。

下面将分别使用继承`Cell`和继承`LossBase`的方法，来定义平均绝对误差损失函数(Mean Absolute Error，MAE)，MAE算法的公式如下所示：

$$ loss= \frac{1}{m}\sum_{i=1}^m\lvert y_i-f(x_i) \rvert$$

上式中$f(x)$为预测值，$y$为样本真实值，$loss$为预测值与真实值之间距离的平均值。

### 基于`nn.Cell`构造损失函数

`nn.Cell`是MindSpore的基类，不但可用于构建网络，还可用于定义损失函数。使用`nn.Cell`定义损失函数的过程与定义一个普通的网络相似，差别在于，其执行逻辑部分要计算的是前向网络输出与真实值之间的误差。

下面展示怎样基于`nn.Cell`自定义损失函数`MAELoss`：

In [2]:
from mindspore import ops
import mindspore as ms

class MAELoss(nn.Cell):
    """自定义损失函数MAELoss"""
    def construct(self, base, target):
        return ops.abs(base - target).mean()

loss = MAELoss()

input_data = Tensor(np.array([0.1, 0.2, 0.3]).astype(np.float32))  # 生成预测值
target_data = Tensor(np.array([0.1, 0.2, 0.2]).astype(np.float32)) # 生成真实值

output = loss(input_data, target_data)
print(output)

0.033333335


### 基于`nn.LossBase`构造损失函数

基于[nn.LossBase](https://www.mindspore.cn/docs/zh-CN/r2.0.0-alpha/api_python/nn/mindspore.nn.LossBase.html#mindspore.nn.LossBase)构造损失函数`MAELoss`与基于`nn.Cell`构造损失函数的过程类似，都要重写`__init__`方法和`construct`方法。

`nn.LossBase`可使用方法`get_loss`将`reduction`应用于损失计算。

In [3]:
class MAELoss(nn.LossBase):
    """自定义损失函数MAELoss"""
    def construct(self, base, target):
        x = ops.abs(base - target)
        return self.get_loss(x)  # 返回loss均值

loss = MAELoss()

input_data = Tensor(np.array([0.1, 0.2, 0.3]).astype(np.float32))  # 生成预测值
target_data = Tensor(np.array([0.1, 0.2, 0.2]).astype(np.float32))  # 生成真实值

output = loss(input_data, target_data)
print(output)

0.033333335


## 损失函数与模型训练

损失函数`MAELoss`自定义完成后，可使用MindSpore的接口[Model](https://www.mindspore.cn/docs/zh-CN/r2.0.0-alpha/api_python/train/mindspore.train.Model.html#mindspore.train.Model)中`train`接口进行模型训练，构造`Model`时需传入前向网络、损失函数和优化器，`Model`会在内部将它们关联起来，生成一个可用于训练的网络模型。

在`Model`中，前向网络和损失函数通过[nn.WithLossCell](https://www.mindspore.cn/docs/zh-CN/r2.0.0-alpha/api_python/nn/mindspore.nn.WithLossCell.html#mindspore.nn.WithLossCell)关联起来，`nn.WithLossCell`支持两个输入，分别为`data`和`label`。

In [4]:
from mindspore.train import Model, LossMonitor
from mindspore.dataset import GeneratorDataset

def get_data(num, w=2.0, b=3.0):
    """生成数据及对应标签"""
    for _ in range(num):
        x = np.random.uniform(-10.0, 10.0)
        noise = np.random.normal(0, 1)
        y = x * w + b + noise
        yield np.array([x]).astype(np.float32), np.array([y]).astype(np.float32)

def create_dataset(num_data, batch_size=16):
    """加载数据集"""
    dataset = GeneratorDataset(list(get_data(num_data)), column_names=['data', 'label'])
    dataset = dataset.batch(batch_size)
    return dataset

train_dataset = create_dataset(num_data=160)
network = nn.Dense(1, 1)
loss_fn = MAELoss()
optimizer = nn.Momentum(network.trainable_params(), learning_rate=0.005, momentum=0.9)

# 使用model接口将网络、损失函数和优化器关联起来
model = Model(network, loss_fn, optimizer)
model.train(10, train_dataset, callbacks=[LossMonitor(10)])

epoch: 1 step: 10, loss is 6.525373935699463
epoch: 2 step: 10, loss is 4.005467414855957
epoch: 3 step: 10, loss is 2.1115174293518066
epoch: 4 step: 10, loss is 2.7334954738616943
epoch: 5 step: 10, loss is 1.7042752504348755
epoch: 6 step: 10, loss is 1.6317998170852661
epoch: 7 step: 10, loss is 1.035435438156128
epoch: 8 step: 10, loss is 0.6060740351676941
epoch: 9 step: 10, loss is 1.0374044179916382
epoch: 10 step: 10, loss is 0.736151397228241


## 多标签损失函数与模型训练

上述定义了一个简单的平均绝对误差损失函数`MAELoss`，但许多深度学习应用的数据集较复杂，如目标检测网络Faster R-CNN的数据中就包含多个标签，而非简单的一条数据对应一个标签，这时损失函数的定义和使用略有不同。

本节介绍在多标签数据集场景下，如何定义多标签损失函数（Multi label loss function），并使用Model进行模型训练。

### 多标签数据集

下例通过`get_multilabel_data`函数拟合两组线性数据$y1$和$y2$，拟合的目标函数为：

$$f(x)=2x+3$$

最终数据集应随机分布于函数周边，这里按以下公式的方式生成，其中`noise`为服从标准正态分布的随机值。`get_multilabel_data`函数返回数据$x$、$y1$和$y2$：

$$f(x)=2x+3+noise$$

通过`create_multilabel_dataset`生成多标签数据集，并将`GeneratorDataset`中的`column_names`参数设置为['data', 'label1', 'label2']，最终返回的数据集格式为一条数据`data`对应两个标签`label1`和`label2`。

In [5]:
def get_multilabel_data(num, w=2.0, b=3.0):
    for _ in range(num):
        x = np.random.uniform(-10.0, 10.0)
        noise1 = np.random.normal(0, 1)
        noise2 = np.random.normal(-1, 1)
        y1 = x * w + b + noise1
        y2 = x * w + b + noise2
        yield np.array([x]).astype(np.float32), np.array([y1]).astype(np.float32), np.array([y2]).astype(np.float32)

def create_multilabel_dataset(num_data, batch_size=16):
    dataset = GeneratorDataset(list(get_multilabel_data(num_data)), column_names=['data', 'label1', 'label2'])
    dataset = dataset.batch(batch_size)  # 每个batch有16个数据
    return dataset

### 多标签损失函数

针对上一步创建的多标签数据集，定义多标签损失函数`MAELossForMultiLabel`。

$$ loss1= \frac{1}{m}\sum_{i=1}^m\lvert y1_i-f(x_i) \rvert$$

$$ loss2= \frac{1}{m}\sum_{i=1}^m\lvert y2_i-f(x_i) \rvert$$

$$ loss = \frac{(loss1 + loss2)}{2}$$

上式中，$f(x)$ 为样例标签的预测值，$y1$ 和 $y2$ 为样例标签的真实值，$loss1$ 为预测值与真实值 $y1$ 之间距离的平均值，$loss2$ 为预测值与真实值 $y2$ 之间距离的平均值 ，$loss$ 为损失值 $loss1$ 与损失值 $loss2$ 平均值。

在`MAELossForMultiLabel`中的`construct`方法的输入有三个，预测值`base`，真实值`target1`和`target2`，在`construct`中分别计算预测值与真实值`target1`、预测值与真实值`target2`之间的误差，将两误差取平均后作为最终的损失函数值。

示例代码如下：

In [6]:
class MAELossForMultiLabel(nn.LossBase):

    def construct(self, base, target1, target2):
        x1 = ops.abs(base - target1)
        x2 = ops.abs(base - target2)
        return (self.get_loss(x1) + self.get_loss(x2)) / 2

### 多标签模型训练


使用Model连接前向网络、多标签损失函数和优化器时，`Model`的网络`network`指定为自定义的损失网络`loss_net`，损失函数`loss_fn`不指定，优化器仍使用`Momentum`。

在未指定`loss_fn`时，`Model`会默认`network`内部已实现损失函数的逻辑，不再在内部使用`nn.WithLossCell`关联前向网络和损失函数。

In [7]:
train_dataset = create_multilabel_dataset(num_data=160)

# 定义多标签损失函数
loss_fn = MAELossForMultiLabel()
# 定义优化器
opt = nn.Momentum(network.trainable_params(), learning_rate=0.005, momentum=0.9)

In [8]:
def train(model, dataset, loss_fn, optimizer):
    # Define forward function
    def forward_fn(data, label1, label2):
        output = network(data)
        return loss_fn(output, label1, label2)

    # Get gradient function
    grad_fn = ms.value_and_grad(forward_fn, None, optimizer.parameters)

    # Define function of one-step training
    def train_step(data, label1, label2):
        loss, grads = grad_fn(data, label1, label2)
        loss = ops.depend(loss, optimizer(grads))
        return loss

    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label1, label2) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label1, label2)

        if batch % 2 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(network, train_dataset, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
loss: 0.739832  [  0/ 10]
loss: 0.949316  [  2/ 10]
loss: 1.052085  [  4/ 10]
loss: 0.982260  [  6/ 10]
loss: 0.784400  [  8/ 10]
Epoch 2
-------------------------------
loss: 0.963160  [  0/ 10]
loss: 0.899232  [  2/ 10]
loss: 0.934914  [  4/ 10]
loss: 0.757601  [  6/ 10]
loss: 0.965961  [  8/ 10]
Epoch 3
-------------------------------
loss: 0.815042  [  0/ 10]
loss: 0.999898  [  2/ 10]
loss: 1.008266  [  4/ 10]
loss: 1.024307  [  6/ 10]
loss: 0.798073  [  8/ 10]
Epoch 4
-------------------------------
loss: 0.844747  [  0/ 10]
loss: 0.958094  [  2/ 10]
loss: 0.898447  [  4/ 10]
loss: 0.879910  [  6/ 10]
loss: 0.969592  [  8/ 10]
Epoch 5
-------------------------------
loss: 0.917983  [  0/ 10]
loss: 0.862990  [  2/ 10]
loss: 0.947069  [  4/ 10]
loss: 0.854086  [  6/ 10]
loss: 0.910622  [  8/ 10]
Done!


本章节简单讲解了多标签数据集场景下，如何定义损失函数并进行模型训练。在很多其他场景中，也可采用此类方法进行模型训练。